In [1]:
import math
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

# **Часть первая**
**Я тут не буду брать в расчет вес рюкзака, время. Рассмотрим работу различных алгоритмов**

In [2]:
class Courier:
    def __init__(self, ln, wd):
        self.pos = [random.randint(0, wd), random.randint(0, ln)]
        self.price = random.randint(25, 40)

    def __str__(self):
        return 'Courier on {} {} cell, his price is {}'.format(self.pos[0], self.pos[1], self.price)


class Order:
    def __init__(self, pos1, pos2, price):
        self.pos1 = pos1
        self.pos2 = pos2
        self.price = price

    def __str__(self):
        return 'Order at ({},{}) cell, the point is at ({},{}), price is {}'.format(self.pos1[0], self.pos1[1], self.pos2[0], self.pos2[1], self.price)

**По ходу наптсания кода, функции будут изменяться с добавлением параметров. Точно так же и с классами Заказа и Курьера**

In [3]:
def dist_count(pos1, pos2):
    dist = math.sqrt((pos1[0] - pos2[0]) ** 2 + (pos1[1] - pos2[1]) ** 2)
    return dist


def income_count(order, courier):
    dst = dist_count(courier.pos, order.pos1) + dist_count(order.pos1, order.pos2)
    income = order.price - courier.price * dst / 10
    return income


def order_parse(file_name):
    raf = []
    file = open(file_name).readlines()
    for i in range(len(file)):
        buf = file[i].split(' ')
        buf[0] = list(map(int, buf[0].replace('(', '').replace(')', '').split(',')))
        buf[1] = list(map(int, buf[1].replace('(', '').replace(')', '').split(',')))
        buf[2] = int(buf[2])
        raf.append(Order(buf[0], buf[1], buf[2]))
    return raf

**Сделаем 9 курьеров на 7 заказов**

In [12]:
random.seed(43)

orders = []
workers = []

for i in range(9):
        workers.append(Courier(100, 100))

orders = order_parse('orders_v1.txt')

In [13]:
arr = np.array([0 for n in range(len(workers)*len(orders))]).reshape(len(workers), len(orders))

for i in range(len(workers)):
    buf = np.array([0 for n in range(len(orders))])
    for j in range(len(orders)):
        buf[j] = income_count(orders[j], workers[i])
    arr[i] = buf

df = pd.DataFrame(arr)
df

,0,1,2,3,4,5,6
0,391,354,-30,-11,47,576,418
1,395,248,77,75,-7,569,437
2,182,46,-196,-107,-188,372,300
3,246,110,123,10,-147,497,324
4,254,96,-44,0,-138,456,355
5,463,316,141,139,51,649,480
6,351,359,-109,-39,35,586,406
7,423,378,17,28,74,644,441
8,339,195,24,21,-57,512,388


**Таблица с моим заработком в случае отдачи заказа тому или иному курьеру**

In [14]:
res = 0

def sort_out(free_couriers, task, income):
    global res
    for n in range(len(free_couriers)):
        income += df[task][free_couriers[n]]
        buf = free_couriers.copy()
        buf.remove(free_couriers[n])
        if task + 1 < len(orders):
            sort_out(buf, task+1, income)
        if task + 1 == len(orders):
            res = max(income, res)
        income -= df[task][free_couriers[n]]

sort_out([i for i in range(len(workers))], 0, 0)
print(res)

2099


**Штука считает все верно, но если в компании 2500 курьеров и 5000 заказов, то она умрет**

In [15]:
res = 0

def greedy_algorithm(df):
    global res
    rows = []
    columns = []
    df = df.reset_index()
    for c in df.columns:
        # print(c)
        r = list(df[c]).index(max(df[c]))
        # print(r, c)
        if r not in rows:
            rows.append(r)
            columns.append(c)
        else:
            # print(-1)
            if df[c][r] > df[columns[rows.index(r)]][r]:
                columns[rows.index(r)] = c
    # print(rows, columns)
    for r, c in zip(rows, columns):
        res += df[c][r]
        df = df.drop([c], axis=1)
        df = df.drop([r], axis=0)
    if not df.empty:
        # print(df[1][3])
        # print(df)
        greedy_algorithm(df)

greedy_algorithm(df)
print(res)

1904


**Как видно из показанного примера, жадный алгоритм быстрее, но менее эффективный, алгоритм перебора нашёл самый оптимальный способ, но это было дольше всего**

# **Часть вторая**

In [16]:
class Agent:
    def __init__(self):
        self.num = None
        self.partner = None
        self.income = 0
    
    def update(self, other, st=0):
        '''
        Создаем пару Курьер-Заказ
        '''
        self.partner = other
        self.other = other
        if st == 0:
            other.update(self, st=1)

    def count(self, other):
        '''
        Для автоматического подсчета выручки
        '''
        self.income = income_count(other, self)
        return self.income

    def clear(self, st=0):
        '''
        Для разрыва связи Курьер-Заказ, на случай если агент нашёл лучшие условия
        '''
        buf = self.other
        if self.partner:
            self.partner = None
            if st==0:
                self.other.clear(st=1)
        return buf
    
    def delivered(self):
        '''
        В случае, когда заказ доставлен и курьер должен остаться на рабочем месте, а заказ идет в сброс
        '''        
        if self.partner:
            self.partner = None
            del self.other
    

class Courier(Agent):
    def __init__(self, ln, wd, num):
        super().__init__()
        self.pos = [random.randint(0, wd), random.randint(0, ln)]
        self.price = random.randint(25, 40)
        self.num = num

    def __str__(self):
        return 'Courier-{} took order number {}'.format(self.num, self.partner.num)


class Order(Agent):
    def __init__(self, pos1, pos2, price, num=0):
        super().__init__()
        self.pos1 = pos1
        self.pos2 = pos2
        self.price = price
        self.num = num

    def __str__(self):
        return 'Order number {}'.format(self.num)

**Тут я написал агента, который отслеживает статус Курьера и заказа**

Идея в том, что теперь каждый курьер и заказ имеют связь друг с другом и в любой момент ее можно прервать (отдать курьеру другой заказ)

In [17]:
def order_parse(file_name):
    raf = []
    file = open(file_name).readlines()
    for i in range(len(file)):
        buf = file[i].split(' ')
        buf[0] = list(map(int, buf[0].replace('(', '').replace(')', '').split(',')))
        buf[1] = list(map(int, buf[1].replace('(', '').replace(')', '').split(',')))
        buf[2] = int(buf[2])
        raf.append(Order(buf[0], buf[1], buf[2], i))
    return raf

In [20]:
random.seed(43)

orders = []
workers = []

for i in range(9):
        workers.append(Courier(100, 100, i))

orders = order_parse('orders_v1.txt')

In [21]:
received_orders = []

for i in range(len(orders)):
    maxi = 0
    obj = None
    for j in range(len(workers)):
        if workers[j].count(orders[i]) > maxi:
            if not workers[j].partner:
                maxi = workers[j].count(orders[i])
                obj = workers[j]
            else:
                if workers[j].income < workers[j].count(orders[i]):
                    order = worker[j].clear()
                    maxi = workers[j].count(orders[i])
                    obj = workers[j]
                    received_ordeers.remove(order)   # Убираем заказ из списка принятых и продолжаем искать вариаеты
    if obj:         
        orders[i].update(obj)
        received_orders.append(orders[i])   # Добавляем заказ в список принятых

amount = 0

for i in workers:
    if i.partner:
        print(i)
        amount += i.income
    else:
        print(None)

# for i in received_orders:
#     print(i)

print('my income is {}'.format(amount))

for i in workers:
    i.delivered()   # прерываю связь всех курьеров с заказами, а заказы удаляю

Courier-0 took order number 4
Courier-1 took order number 3
None
Courier-3 took order number 2
None
Courier-5 took order number 0
Courier-6 took order number 5
Courier-7 took order number 1
Courier-8 took order number 6
my income is 2896.9277903707966


**Новая идея - соеденить агентов с алгоритмами, чтобы максимизировать выручку**

**Так же рассмотрим другие алгоритмы распределения заказов - тот же Форд-Фолкирсон**

# **Часть третья**